In [ ]:
import os
import sys

path_1 = os.path.dirname(os.getcwd())
path_2 = path_1 + '/' + 'CommonCodes/'
sys.path.insert(0, path_2)

import numpy as np
import pandas as pd
import HelperFunctions as helper

import warnings
warnings.filterwarnings('ignore')

In [ ]:
N = 100 
feature_selection_repetition = 1000

for n_r in range(feature_selection_repetition):

    train_datasets_names = ['GSE16879', 'GSE22619', 'GSE59071', 'GSE102133', 'GSE179285']
    test_datasets_names = ['GSE9452', 'GSE36807' ,'GSE37283', 'GSE4183']

    _test_datasets = []
    for d in test_datasets_names:
        _test_datasets.append(pd.read_pickle(path_1+'/Datasets/'+d))
    
    _train_datasets = []
    for d in train_datasets_names:
        _train_datasets.append(pd.read_pickle(path_1+'/Datasets/'+d))

    for i in range(len(_test_datasets)):
        _test_datasets[i] = _test_datasets[i].groupby(_test_datasets[i].columns, axis=1).agg(np.mean)

    for i in range(len(_train_datasets)):
        _train_datasets[i] = _train_datasets[i].groupby(_train_datasets[i].columns, axis=1).agg(np.mean)

    common_cols = list(set.intersection(*(set(df.columns) for df in [*_train_datasets, *_test_datasets])))
    for i in range(len(_train_datasets)):
        _train_datasets[i] = _train_datasets[i][common_cols]
        y = _train_datasets[i].label
        _train_datasets[i].drop(['label'], axis=1, inplace=True)
        _train_datasets[i]['label'] = y
        _train_datasets[i].columns = np.arange(len(_train_datasets[i].columns))

    df_train = pd.concat(_train_datasets)

    df_train = helper.oversample(df_train).sample(frac=1).reset_index(drop=True)

    x_train, y_train = helper.datasetXY_S(df_train)

    features = helper.SVFS(x_train, y_train, th_irr=3, diff_threshold=1.7, th_red=4, k=N, alpha=50, beta=5)
    features = [common_cols[f] for f in features]

    helper.writeToFile('IBD_features.txt', features)
    print(f'round: {n_r}/{feature_selection_repetition}')